In [1]:
%matplotlib inline

import os
import zipfile
import urllib

DOWNLOAD_ROOT = "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/"
FOOD_PATH = os.path.join("datasets", "openfoodfacts")
FOOD_URL = DOWNLOAD_ROOT + "fr.openfoodfacts.org.products.csv.zip"





# Téléchargement et décompression des données

In [2]:
def fetch_food_data(food_url=FOOD_URL, food_path=FOOD_PATH):
    if not os.path.isdir(food_path):
        os.makedirs(food_path)
    archive_path = os.path.join(food_path, "fr.openfoodfacts.org.products.csv.zip")
    urllib.request.urlretrieve(food_url, archive_path)
    food_archive = zipfile.ZipFile(archive_path)
    food_archive.extractall(path=food_path)
    food_archive.close()

In [3]:
fetch_food_data()

# Import du fichier CSV

In [6]:
import pandas as pd

def load_food_data(food_path=FOOD_PATH):
    csv_path = os.path.join(food_path, "fr.openfoodfacts.org.products.csv")
    return pd.read_csv(csv_path)

food = load_food_data()
food.head()

ParserError: Error tokenizing data. C error: Expected 34 fields in line 3, saw 37


## Le chargement a échoué à cause d'un problème de format sur une ligne.
## Inspection de quelques lignes du fichier pour avoir un aperçu visuel du texte brut :

In [14]:
def read_raw_file(nblines, food_path = FOOD_PATH):
    csv_path = os.path.join(food_path, "fr.openfoodfacts.org.products.csv")
    
    fp = open(csv_path)
    
    line = ""
    
    for cnt_lines in range(nblines+1):
        line = fp.readline()
        
    print(">>>>>> Line %d" % (cnt_lines))
    print(line)
    
    

In [16]:
read_raw_file(0)

>>>>>> Line 0
code	url	creator	created_t	created_datetime	last_modified_t	last_modified_datetime	product_name	generic_name	quantity	packaging	packaging_tags	brands	brands_tags	categories	categories_tags	categories_fr	origins	origins_tags	manufacturing_places	manufacturing_places_tags	labels	labels_tags	labels_fr	emb_codes	emb_codes_tags	first_packaging_code_geo	cities	cities_tags	purchase_places	stores	countries	countries_tags	countries_fr	ingredients_text	allergens	allergens_fr	traces	traces_tags	traces_fr	serving_size	no_nutriments	additives_n	additives	additives_tags	additives_fr	ingredients_from_palm_oil_n	ingredients_from_palm_oil	ingredients_from_palm_oil_tags	ingredients_that_may_be_from_palm_oil_n	ingredients_that_may_be_from_palm_oil	ingredients_that_may_be_from_palm_oil_tags	nutrition_grade_uk	nutrition_grade_fr	pnns_groups_1	pnns_groups_2	states	states_tags	states_fr	main_category	main_category_fr	image_url	image_small_url	energy_100g	energy-from-fat_100g	fat_100g	saturated-

In [19]:
read_raw_file(1)
read_raw_file(2)
read_raw_file(3)
read_raw_file(4)
read_raw_file(5)

>>>>>> Line 1
0000000003087	http://world-fr.openfoodfacts.org/produit/0000000003087/farine-de-ble-noir-ferme-t-y-r-nao	openfoodfacts-contributors	1474103866	2016-09-17T09:17:46Z	1474103893	2016-09-17T09:18:13Z	Farine de blé noir		1kg			Ferme t'y R'nao	ferme-t-y-r-nao																		en:FR	en:france	France																							en:to-be-completed, en:nutrition-facts-to-be-completed, en:ingredients-to-be-completed, en:expiration-date-to-be-completed, en:characteristics-to-be-completed, en:categories-to-be-completed, en:brands-completed, en:packaging-to-be-completed, en:quantity-completed, en:product-name-completed, en:photos-to-be-validated, en:photos-uploaded	en:to-be-completed,en:nutrition-facts-to-be-completed,en:ingredients-to-be-completed,en:expiration-date-to-be-completed,en:characteristics-to-be-completed,en:categories-to-be-completed,en:brands-completed,en:packaging-to-be-completed,en:quantity-completed,en:product-name-completed,en:photos-to-be-validated,en:photos-uploaded	A com

### On voit que le séparateur des données semble être la tabulation.
De plus, dans la documentation des différents champs qui a été fournie, il est indiqué d'utiliser le séparateur tabulation et l'encodage UTF-8 :
https://world.openfoodfacts.org/data/data-fields.txt

### On fait donc une nouvelle tentative de chargement en spécifiant le séparateur tabulation, avec encodage utf-8


In [29]:
import pandas as pd

def load_food_data(food_path=FOOD_PATH):
    csv_path = os.path.join(food_path, "fr.openfoodfacts.org.products.csv")
    return pd.read_csv(csv_path, sep='\t', header=0, encoding='utf-8')



### Puis on affiche quelques instances de données :

In [30]:
food = load_food_data()
food.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_fr,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_fr,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_fr,ingredients_text,allergens,allergens_fr,traces,traces_tags,traces_fr,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_fr,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_fr,main_category,main_category_fr,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,omega-6-fat_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,3087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","A compléter,Informations nutritionnelles à com...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,États-Unis,"Bananas, vegetable oil (coconut oil, corn oil ...",NaN,NaN,NaN,NaN,NaN,28 g (1 ONZ),NaN,0.0,[ bananas -> en:bananas ] [ vegetable-oil -

# Correction d'une limitation de volume sur jupyter notebook

Lors de la première tentativement de chargement, j'ai eu l'erreur suivante :

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)


J'ai donc redémarré le jupyter notebook comme suit :
$ jupyter notebook --generate-config
Puis j'ai édité le fichier jupyter_notebook_config.py pour changer la valeur comme suit :

c.NotebookApp.iopub_data_rate_limit = 10000000000

Et j'ai relancé jupyoter notebook
Ensuite, j'ai pu relancer load_food_data() dans la cellule ci-dessus

# Inspection générale des données

In [24]:
pd.options.display.max_columns = 1000

In [32]:
food.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_fr,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_fr,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_fr,ingredients_text,allergens,allergens_fr,traces,traces_tags,traces_fr,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_fr,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_fr,main_category,main_category_fr,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,omega-6-fat_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,3087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","A compléter,Informations nutritionnelles à com...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,États-Unis,"Bananas, vegetable oil (coconut oil, corn oil ...",NaN,NaN,NaN,NaN,NaN,28 g (1 ONZ),NaN,0.0,[ bananas -> en:bananas ] [ vegetable-oil -

In [35]:
food.tail()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_fr,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_fr,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_fr,ingredients_text,allergens,allergens_fr,traces,traces_tags,traces_fr,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_fr,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_fr,main_category,main_category_fr,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,omega-6-fat_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
320767,9948282780603,http://world-fr.openfoodfacts.org/produit/9948...,openfoodfacts-contributors,1490631299,2017-03-27T16:14:59Z,1491244498,2017-04-03T18:34:58Z,Tomato & ricotta,NaN,1,Plastique,plastique,Panzani,panzani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:RO,en:romania,Roumanie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","A compléter,Informations nutritionnelles à com...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320768,99567453,http://world-fr.openfoodfacts.org/produit/9956...,usda-ndb-import,1489059076,2017-03-09T11:31:16Z,1491244499,2017-04-03T18:34:59Z,"Mint Melange Tea A Blend Of Peppermint, Lemon ...",NaN,NaN,Plastique,plastique,Trader Joe's,trader-joe-s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,États-Unis,"Organic peppermint, organic lemon grass, organ...",NaN,NaN,NaN,NaN

## Affichage des colonnes :

In [43]:
food.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Data columns (total 162 columns):
code                                          320749 non-null object
url                                           320749 non-null object
creator                                       320770 non-null object
created_t                                     320769 non-null object
created_datetime                              320763 non-null object
last_modified_t                               320772 non-null object
last_modified_datetime                        320772 non-null object
product_name                                  303010 non-null object
generic_name                                  52795 non-null object
quantity                                      104819 non-null object
packaging                                     78960 non-null object
packaging_tags                                78961 non-null object
brands                                        292360 non-null o

In [36]:
food.describe()

,no_nutriments,additives_n,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,nutrition_grade_uk,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,omega-6-fat_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
count,0.0,248939.000000,248939.000000,0.0,248939.000000,0.0,0.0,2.611130e+05,857.000000,243891.000000,229554.000000,0.0,0.0,1.0,2.000000,4.000000,1.0,1.0,1.0,24.000000,23.000000,0.0,0.0,1.0,0.0,22823.000000,22859.000000,841.000000,186.000000,38.000000,78.000000,188.000000,149.000000,8.000000,24.000000,23.000000,21.000000,13.000000,0.0,1.400000e+01,0.0,0.0,0.0,143298.000000,144090.000000,243588.000000,244971.000000,72.000000,26.000000,38.000000,262.000000,4.000000,11.000000,266.000000,414.000000,200886.000000,259922.000000,27.000000,16.000000,9.000000,255510.000000,255463.000000,4133.000000,137554.000000,34.000000,7057.000000,1340.000000,918.000000,140867.000000,11154.000000,10815.000000,11729.000000,6784.000000,5240.000000,3042.000000,5.300000e+03,330.000000,2483.000000,38.000000,81.000000,24748.000000,158.000000,141050.000000,5845.000000,140462.000000,6253.000000,3929.000000,2106.000000,1620.000000,79.000000,1168.000000,20.000000,11.000000,259.000000,78.000000,29.000000,49.000000,3036.000000,165.000000,948.000000,0.0,268.000000,221210.000000,221210.000000,0.0,0.0
mean,NaN,1.936024,0.019659,NaN,0.055246,NaN,NaN,1.141915e+03,585.501214,12.730379,5.129932,NaN,NaN,7.4,6.040000,36.136182,18.9,8.1,3.0,10.752667,10.673913,NaN,NaN,61.0,NaN,10.425055,6.312493,3.182103,2.250285,3.186553,1.635462,16.229144,3.823819,0.057000,0.153842,0.061567,40.192857,25.123077,NaN,1.357143e-06,NaN,NaN,NaN,0.073476,0.020071,32.073981,16.003484,11.841667,2.878846,25.897368,6.354084,24.325000,12.163636,31.925000,49.790389,2.862111,7.075940,4.658148,2.506250,0.021678,2.028624,0.798815,7.838105,0.000397,0.518715,0.000008,0.056705,0.034219,0.023367,0.325574,0.259007,0.020303,0.023378,0.006898,0.205856,8.938696e-05,0.121290,0.072138,0.013123,0.119052,0.424635,0.092638,0.125163,0.617282,0.003652,0.534143,0.007950,0.025794,0.003014,0.012161,0.003126,0.001690,0.000401,0.000427,1.594563,0.145762,6.425698,31.458587,15.412121,49.547785,NaN,341.700764,9.165535,9.058049,NaN,NaN
std,NaN,2.502019,0.140524,NaN,0.269207,NaN,NaN,6.447154e+03,712.809943,17.578747,8.014238,NaN,NaN,NaN,0.226274,24.101433,NaN,NaN,NaN,4.019993,3.379647,NaN,NaN,NaN,NaN,17.0761

### Premiers constats :
#### De nombreux champs ont très peu de valeurs renseignées, voire aucune
#### Certaines colonnes sont présentes dans le fichier CSV mais pas dans l'url de description des champs fournie :

allergens 28344 non-null object

allergens_fr 19 non-null object

traces_fr 24352 non-null object

additives_fr 154680 non-null object

nutrition_grade_uk (mais 0 valeurs sont présentes)

pnns_groups_1  91513 non-null object

pnns_groups_2  94491 non-null object

states         320726 non-null object

states_tags    320726 non-null object

states_fr      320726 non-null object      



### Des url sont fournies pour les produits.  Voir si on pourra compléter les données avec les url fournies :

In [61]:
pd.options.display.max_colwidth = 100
food['url']

0                  http://world-fr.openfoodfacts.org/produit/0000000003087/farine-de-ble-noir-ferme-t-y-r-nao
1                        http://world-fr.openfoodfacts.org/produit/0000000004530/banana-chips-sweetened-whole
2                                http://world-fr.openfoodfacts.org/produit/0000000004559/peanuts-torn-glasser
3                    http://world-fr.openfoodfacts.org/produit/0000000016087/organic-salted-nut-mix-grizzlies
4                      http://world-fr.openfoodfacts.org/produit/0000000016094/organic-polenta-bob-s-red-mill
5              http://world-fr.openfoodfacts.org/produit/0000000016100/breadshop-honey-gone-nuts-granola-unfi
6              http://world-fr.openfoodfacts.org/produit/0000000016117/organic-long-grain-white-rice-lundberg
7                       http://world-fr.openfoodfacts.org/produit/0000000016124/organic-muesli-daddy-s-muesli
8         http://world-fr.openfoodfacts.org/produit/0000000016193/organic-dark-chocolate-minis-equal-exchange
9         

### Un clic sur un ensemble d'url ci-dessus montre qu'elles ne fonctionnent pas : on ne pourra donc pas y récupérer des informations additionnelles
Pour aller plus loin, on note que la colonne "creator" contient l'origine des données : on pourra donc remonter jusqu'à l'origine de ces données pour essayer d'avoir plus d'informations si nécessaire.

Pour l'instant, on ne va pas creuser cet aspect qui risque de prendre beaucoup de temps, et on va d'abord se concentrer sur les données qui sont déjà à notre disposition.

## Analyse plus détaillée des données et des colonnes manquantes
Le code source de cette analyse du pourcentage de colonnes manquantes a été trouvé sur : 
https://www.kaggle.com/daveianhickey/can-we-trust-open-food-facts



In [148]:
def analyse_donnees_manquantes(df):
    cols = df.columns
    low_data_cols = []
    missing_pct = {}

    for c in cols:
        empty_pct = round(len(df[pd.isnull(df[c])])/df.shape[0],2)
        if empty_pct >= 0.7:
            missing_pct[c]=empty_pct
            low_data_cols.append(c)

    low = round((len(low_data_cols)/len(cols))*100,2)
    print(low,"% des colonnes ont >=70% de données manquantes. \nCes colonnes sont:")
    #print(len(food_fr['creator'].isnull()))
    print(missing_pct)
    
analyse_donnees_manquantes(food)

73.46 % des colonnes ont >=70% de données manquantes. 
Ces colonnes sont:
{'generic_name': 0.84, 'packaging': 0.75, 'packaging_tags': 0.75, 'categories': 0.74, 'categories_tags': 0.74, 'categories_fr': 0.74, 'origins': 0.93, 'origins_tags': 0.93, 'manufacturing_places': 0.89, 'manufacturing_places_tags': 0.89, 'labels': 0.85, 'labels_tags': 0.85, 'labels_fr': 0.85, 'emb_codes': 0.91, 'emb_codes_tags': 0.91, 'first_packaging_code_geo': 0.94, 'cities': 1.0, 'cities_tags': 0.94, 'purchase_places': 0.82, 'stores': 0.84, 'allergens': 0.91, 'allergens_fr': 1.0, 'traces': 0.92, 'traces_tags': 0.92, 'traces_fr': 0.92, 'no_nutriments': 1.0, 'ingredients_from_palm_oil': 1.0, 'ingredients_from_palm_oil_tags': 0.98, 'ingredients_that_may_be_from_palm_oil': 1.0, 'ingredients_that_may_be_from_palm_oil_tags': 0.96, 'nutrition_grade_uk': 1.0, 'pnns_groups_1': 0.71, 'pnns_groups_2': 0.71, 'main_category': 0.74, 'main_category_fr': 0.74, 'image_url': 0.76, 'image_small_url': 0.76, 'energy-from-fat_100g'

### On voit que 73.46 % des colonnes ont >=70% de données manquantes

## Filtre sur les produits vendus en France
L'objectif étant de préparer des données pour des recettes pour un site français, on va faire l'hypothèse que les produits de ces recettes sont vendus en France.
=> Poser la question au client pour confirmer

#### Consultation des valeurs des tags des pays pour voir sur quelles valeurs filtrer pour conserver uniquement la France :

In [128]:
food['countries_tags'].value_counts()

en:united-states                                                                                                                            172998
en:france                                                                                                                                    94391
en:switzerland                                                                                                                               14953
en:germany                                                                                                                                    7870
en:spain                                                                                                                                      5009
en:united-kingdom                                                                                                                             4825
en:belgium                                                                                                            

In [129]:
food_fr = food[food['countries_tags'].str.contains("france")==True].copy()

In [130]:
food_fr.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98440 entries, 0 to 320770
Data columns (total 162 columns):
code                                          98440 non-null object
url                                           98440 non-null object
creator                                       98440 non-null object
created_t                                     98440 non-null object
created_datetime                              98439 non-null object
last_modified_t                               98440 non-null object
last_modified_datetime                        98440 non-null object
product_name                                  91247 non-null object
generic_name                                  38291 non-null object
quantity                                      71237 non-null object
packaging                                     58971 non-null object
packaging_tags                                58971 non-null object
brands                                        86426 non-null object
brand

## Nouvelle analyse des données manquantes, avec uniquement les produits vendus en France

In [149]:
analyse_donnees_manquantes(food_fr)

67.28 % des colonnes ont >=70% de données manquantes. 
Ces colonnes sont:
{'origins': 0.84, 'origins_tags': 0.84, 'manufacturing_places': 0.73, 'manufacturing_places_tags': 0.74, 'emb_codes': 0.77, 'emb_codes_tags': 0.77, 'first_packaging_code_geo': 0.81, 'cities': 1.0, 'cities_tags': 0.8, 'allergens': 0.77, 'allergens_fr': 1.0, 'traces': 0.81, 'traces_tags': 0.81, 'traces_fr': 0.81, 'serving_size': 0.73, 'no_nutriments': 1.0, 'ingredients_from_palm_oil': 1.0, 'ingredients_from_palm_oil_tags': 0.96, 'ingredients_that_may_be_from_palm_oil': 1.0, 'ingredients_that_may_be_from_palm_oil_tags': 0.94, 'nutrition_grade_uk': 1.0, 'energy-from-fat_100g': 1.0, 'butyric-acid_100g': 1.0, 'caproic-acid_100g': 1.0, 'caprylic-acid_100g': 1.0, 'capric-acid_100g': 1.0, 'lauric-acid_100g': 1.0, 'myristic-acid_100g': 1.0, 'palmitic-acid_100g': 1.0, 'stearic-acid_100g': 1.0, 'arachidic-acid_100g': 1.0, 'behenic-acid_100g': 1.0, 'lignoceric-acid_100g': 1.0, 'cerotic-acid_100g': 1.0, 'montanic-acid_100g': 1

# Sélection des features à nettoyer et analyser

Vu le nombre important de colonnes disponibles, il va falloir faire une sélection par rapport à l'objectif, c'est à dire par rapport aux principes de base d'une alimentation saine

## Recherche sur les principes de base d'une alimentation saine

#### Consommer un maximum de « raw food » (produits bruts non transformés) avec une priorité absolue sur les fruits et légumes
Eviter les aliments trop sucrés  
privilégier le bio français  (versus bio industriel intensif dont la charte change d'un pays à l'autre)  

Source : https://alimentsain.fr/  


#### manger des produits de saison :
Indiqués ici : https://alimentsain.fr/aliment/calendrier-fruits-legumes/


#### Fruits, aliments riches en fibres, poissons gras (saumon, sardine, maquereau), légumineuses ( : haricots, lentilles, soja, pois entiers ou cassés, pois chiches, fèves, luzerne ou lupins,
), fruits secs, viande blanche, huile végétale

#### Alimentation variée et équilibrée

####  Eviter les additifs alimentaires considérés comme nocifs / cancérigènes

#### Eviter les aliments trop salés

#### Selon l'Organisation mondiale de la santé, un régime alimentaire sain est composé des éléments suivants1 :

    des fruits, des légumes, des légumes secs (comme des lentilles et des pois), des fruits secs et des céréales complètes (par exemple du maïs, millet, avoine, blé et riz brun non transformés)1
    au moins 400 g (soit 5 portions) de fruits et légumes par jour, hors pommes de terre, patates douces, manioc et autres racines amylacées1
    moins de 10 % de l’apport énergique total provenant de sucres libres, soit l’équivalent de 50 g (ou environ 12 cuillères à café rases) pour une personne de poids normal consommant environ 2000 calories par jour ; dans l’idéal, pour préserver davantage la santé, cette part devrait être inférieure à 5 % des apports énergiques totaux1
    moins de 30 % de l’apport énergétique total provenant des matières grasses. Les graisses insaturées (poisson, avocat et noix, et les huiles de tournesol, de soja, de colza et d’olive) sont à préférer aux graisses saturées (viande grasse, beurre, huile de palme et de noix de coco, crème, fromage, beurre clarifié et saindoux) et aux acides gras trans (aliments industriels et viande et produits laitiers provenant des animaux ruminants). Il est proposé de réduire l’apport en graisses saturées à moins de 10 % de l’apport énergétique total et celui en acides gras trans à moins de 1 % ; les acides gras trans sont à exclure d'une alimentation saine1.
    moins de 5 g de sel (soit environ une cuillère à café) par jour, de préférence iodé1

Source : Wikipedia


####  AJR (Apports Journaliers Recommandés) en France :

Nutriment 	Apport journalier recommandé  
Vitamine A (rétinol) 	800 μg  
Vitamine B1 (thiamine) 	1,1 mg  
Vitamine B2 (riboflavine) 	1,4 mg  
Vitamine B3 (ou PP, niacine) 	16 mg  
Vitamine B5 (acide pantothénique) 	6 mg  
Vitamine B6 (pyridoxine) 	1,4 mg  
Vitamine B8 ou H (biotine) 	50 μg  
Vitamine B9 (acide folique)(Folacine) 	200 μg  
Vitamine B12 (cobalamine) 	2,5 μg  
Vitamine C (acide ascorbique) 	80 mg  
Vitamine D (cholécalciférol) 	5 μg  
Vitamine E (tocophérol) 	12 mg  
Vitamine K (anti-AVK) 	75 μg  
Calcium 	800 mg  
Fer 	14 mg  
Iode 	150 μg  
Magnésium 	375 mg  
Phosphore 	700 mg  
Sélénium 	55 μg  
Zinc 	10 mg  
Potassium 	2 000 mg  
Chlorure 	800 mg  
Cuivre 	1 mg  
Manganèse 	2 mg  
Fluorure 	3,5 mg  
Chrome 	40 μg  
Molybdène 	50 μg  

Source: Wikipedia : https://fr.wikipedia.org/wiki/Apports_journaliers_recommand%C3%A9s

In [156]:
food['main_category_fr'].value_counts()

Boissons                                                               6054
Epicerie                                                               2902
Chocolats                                                              2789
Aliments et boissons à base de végétaux                                2745
Conserves                                                              2519
Biscuits                                                               2097
Plats préparés                                                         2079
Snacks sucrés                                                          1946
Surgelés                                                               1889
Petit-déjeuners                                                        1793
Fromages                                                               1785
Pains                                                                  1771
Desserts                                                               1684
Pâtes alimen

## Première sélection de features
#### Par rapport aux éléments ci-dessus voici une première sélection des features qui seront importantes à considérer :
Pour aider à la sélection des features, on utilise la cellule juste au dessus pour afficher leurs valeurs possibles.
Par exemple quand on se pose la question de conserver main_category_fr, on peut faire :
food['main_category_fr'].value_counts()

#### Et voici les features que l'on a enlevé :

In [167]:
food_filtered = food.drop(['packaging', 'packaging_tags', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'countries', 'countries_fr','allergens_fr','traces','traces_fr','no_nutriments','serving_size','additives','additives_fr','cocoa_100g','ingredients_from_palm_oil','ingredients_that_may_be_from_palm_oil','nutrition_grade_uk','butyric-acid_100g','caproic-acid_100g','caprylic-acid_100g','capric-acid_100g','lauric-acid_100g','myristic-acid_100g','palmitic-acid_100g','stearic-acid_100g','arachidic-acid_100g','behenic-acid_100g','lignoceric-acid_100g','cerotic-acid_100g','montanic-acid_100g','melissic-acid_100g','omega-3-fat_100g','alpha-linolenic-acid_100g','eicosapentaenoic-acid_100g','docosahexaenoic-acid_100g','omega-6-fat_100g','linoleic-acid_100g','arachidonic-acid_100g','gamma-linolenic-acid_100g','dihomo-gamma-linolenic-acid_100g','omega-9-fat_100g','oleic-acid_100g','elaidic-acid_100g','gondoic-acid_100g','mead-acid_100g','erucic-acid_100g','nervonic-acid_100g','sucrose_100g','glucose_100g','fructose_100g','lactose_100g','maltose_100g','maltodextrins_100g','starch_100g','polyols_100g','casein_100g','serum-proteins_100g','nucleotides_100g','beta-carotene_100g','biotin_100g','silica_100g','bicarbonate_100g','chloride_100g','fluoride_100g', 'chromium_100g','molybdenum_100g','caffeine_100g','taurine_100g','ph_100g','chlorophyl_100g','glycemic-index_100g','water-hardness_100g','collagen-meat-protein-ratio_100g','ingredients_from_palm_oil_tags','ingredients_that_may_be_from_palm_oil_tags','states','states_fr','main_category'], axis=1)
food_filtered_fr = food_fr.drop(['packaging', 'packaging_tags', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'countries', 'countries_fr','allergens_fr','traces','traces_fr','no_nutriments','serving_size','additives','additives_fr','cocoa_100g','ingredients_from_palm_oil','ingredients_that_may_be_from_palm_oil','nutrition_grade_uk','butyric-acid_100g','caproic-acid_100g','caprylic-acid_100g','capric-acid_100g','lauric-acid_100g','myristic-acid_100g','palmitic-acid_100g','stearic-acid_100g','arachidic-acid_100g','behenic-acid_100g','lignoceric-acid_100g','cerotic-acid_100g','montanic-acid_100g','melissic-acid_100g','omega-3-fat_100g','alpha-linolenic-acid_100g','eicosapentaenoic-acid_100g','docosahexaenoic-acid_100g','omega-6-fat_100g','linoleic-acid_100g','arachidonic-acid_100g','gamma-linolenic-acid_100g','dihomo-gamma-linolenic-acid_100g','omega-9-fat_100g','oleic-acid_100g','elaidic-acid_100g','gondoic-acid_100g','mead-acid_100g','erucic-acid_100g','nervonic-acid_100g','sucrose_100g','glucose_100g','fructose_100g','lactose_100g','maltose_100g','maltodextrins_100g','starch_100g','polyols_100g','casein_100g','serum-proteins_100g','nucleotides_100g','beta-carotene_100g','biotin_100g','silica_100g','bicarbonate_100g','chloride_100g','fluoride_100g', 'chromium_100g','molybdenum_100g','caffeine_100g','taurine_100g','ph_100g','chlorophyl_100g','glycemic-index_100g','water-hardness_100g','collagen-meat-protein-ratio_100g','ingredients_from_palm_oil_tags','ingredients_that_may_be_from_palm_oil_tags','states','states_fr','main_category'], axis=1)

In [166]:
food_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Data columns (total 82 columns):
code                                       320749 non-null object
url                                        320749 non-null object
creator                                    320770 non-null object
created_t                                  320769 non-null object
created_datetime                           320763 non-null object
last_modified_t                            320772 non-null object
last_modified_datetime                     320772 non-null object
product_name                               303010 non-null object
generic_name                               52795 non-null object
quantity                                   104819 non-null object
brands                                     292360 non-null object
brands_tags                                292352 non-null object
categories                                 84410 non-null object
categories_tags                  

In [168]:
food_filtered_fr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98440 entries, 0 to 320770
Data columns (total 82 columns):
code                                       98440 non-null object
url                                        98440 non-null object
creator                                    98440 non-null object
created_t                                  98440 non-null object
created_datetime                           98439 non-null object
last_modified_t                            98440 non-null object
last_modified_datetime                     98440 non-null object
product_name                               91247 non-null object
generic_name                               38291 non-null object
quantity                                   71237 non-null object
brands                                     86426 non-null object
brands_tags                                86422 non-null object
categories                                 61955 non-null object
categories_tags                            61

In [169]:
analyse_donnees_manquantes(food_filtered)

56.1 % des colonnes ont >=70% de données manquantes. 
Ces colonnes sont:
{'generic_name': 0.84, 'categories': 0.74, 'categories_tags': 0.74, 'categories_fr': 0.74, 'origins': 0.93, 'origins_tags': 0.93, 'manufacturing_places': 0.89, 'manufacturing_places_tags': 0.89, 'labels': 0.85, 'labels_tags': 0.85, 'labels_fr': 0.85, 'cities_tags': 0.94, 'purchase_places': 0.82, 'stores': 0.84, 'allergens': 0.91, 'traces_tags': 0.92, 'pnns_groups_1': 0.71, 'pnns_groups_2': 0.71, 'main_category_fr': 0.74, 'image_url': 0.76, 'image_small_url': 0.76, 'energy-from-fat_100g': 1.0, 'monounsaturated-fat_100g': 0.93, 'polyunsaturated-fat_100g': 0.93, 'alcohol_100g': 0.99, 'vitamin-d_100g': 0.98, 'vitamin-e_100g': 1.0, 'vitamin-k_100g': 1.0, 'vitamin-b1_100g': 0.97, 'vitamin-b2_100g': 0.97, 'vitamin-pp_100g': 0.96, 'vitamin-b6_100g': 0.98, 'vitamin-b9_100g': 0.98, 'folates_100g': 0.99, 'vitamin-b12_100g': 0.98, 'pantothenic-acid_100g': 0.99, 'potassium_100g': 0.92, 'phosphorus_100g': 0.98, 'magnesium_100g'

In [170]:
analyse_donnees_manquantes(food_filtered_fr)

46.34 % des colonnes ont >=70% de données manquantes. 
Ces colonnes sont:
{'origins': 0.84, 'origins_tags': 0.84, 'manufacturing_places': 0.73, 'manufacturing_places_tags': 0.74, 'cities_tags': 0.8, 'allergens': 0.77, 'traces_tags': 0.81, 'energy-from-fat_100g': 1.0, 'monounsaturated-fat_100g': 0.99, 'polyunsaturated-fat_100g': 0.99, 'trans-fat_100g': 1.0, 'cholesterol_100g': 1.0, 'alcohol_100g': 0.98, 'vitamin-a_100g': 0.99, 'vitamin-d_100g': 0.99, 'vitamin-e_100g': 0.99, 'vitamin-k_100g': 1.0, 'vitamin-c_100g': 0.99, 'vitamin-b1_100g': 0.99, 'vitamin-b2_100g': 0.99, 'vitamin-pp_100g': 0.99, 'vitamin-b6_100g': 0.99, 'vitamin-b9_100g': 0.99, 'folates_100g': 1.0, 'vitamin-b12_100g': 0.99, 'pantothenic-acid_100g': 1.0, 'potassium_100g': 1.0, 'calcium_100g': 0.98, 'phosphorus_100g': 0.99, 'iron_100g': 0.99, 'magnesium_100g': 0.99, 'zinc_100g': 1.0, 'copper_100g': 1.0, 'manganese_100g': 1.0, 'selenium_100g': 1.0, 'iodine_100g': 1.0, 'fruits-vegetables-nuts_100g': 0.97, 'carbon-footprint_10

In [174]:
food_filtered_fr.describe()

,additives_n,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,pantothenic-acid_100g,potassium_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,selenium_100g,iodine_100g,fruits-vegetables-nuts_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
count,53466.000000,53466.000000,53466.000000,6.459300e+04,30.000000,47642.000000,62375.000000,1186.000000,1205.000000,386.000000,415.000000,47211.000000,62515.000000,45723.000000,64318.000000,62574.000000,62571.000000,2418.000000,589.000000,591.000000,791.000000,72.000000,1297.000000,860.000000,646.000000,710.000000,746.000000,690.000000,2.000000,5.550000e+02,4.260000e+02,337.000000,2257.000000,563.000000,1185.000000,815.000000,231.000000,119.000000,108.000000,9.200000e+01,1.210000e+02,2971.000000,176.000000,61415.000000,61415.000000
mean,1.856619,0.069839,0.148281,1.171553e+03,547.471667,13.332232,5.423696,18.574106,9.807383,0.209285,0.041158,27.759277,13.432792,2.559271,7.754531,1.160535,0.456924,9.136038,0.000627,0.000005,0.052269,0.000517,0.125570,0.298463,0.005076,0.067138,0.186858,0.002184,0.000147,7.271086e-04,8.483711e-03,0.911889,0.325133,0.287926,0.044379,0.234942,0.017869,0.319860,0.017846,3.777380e-04,3.501963e-04,31.546439,366.265368,8.683204,8.362810
std,2.561405,0.258593,0.450784,1.283590e+04,574.728341,16.926708,8.531083,21.960039,14.992141,0.984866,0.535806,27.413340,19.087618,4.634788,7.887373,4.309815,1.696759,10.597055,0.005574,0.000018,0.631852,0.003939,2.800254,6.330165,0.061747,0.735266,3.633147,0.033010,0.000046,1.697901e-02,6.781982e-02,4.063764,1.739698,0.382479,0.757976,1.968875,0.185784,3.446577,0.096645,3.131559e-03,8.859487e-04,32.055360,403.553375,9.046131,9.384056
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000003,0.000000,0.000000,0.000000,0.000002,0.000000,0.000007,0.000114,0.000000e+00,9.000000e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000300,0.000022,0.000001,2.500000e-08,7.500000e-07,0.000000,0.000000,-15.000000,-15.000000
25%,0.000000,0.000000,0.000000,4.270000e+02,62.900000,1.300000,0.300000,2.600000,1.500000,0.000000,0.000000,4.000000,1.000000,0.000000,1.800000,0.080000,0.031496,1.000000,0.000058,0.000001,0.001800,0.000004,0.012000,0.000330,0.000320,0.003500,0.000320,0.000050,0.000130,5.850000e-07,9.000000e-04,0.027600,0.120000,0.159000,0.002400,0.044000,0.000950,0.000090,0.000038,4.025000e-06,1.600000e-05,0.000000,149.750000,1.000000,0.000000
50%,1.000000,0.000000,0.000000,1.035000e+03,208.500000,6.800000,2.000000,9.100000,3.500000,0.000000,0.000000,14.500000,4.100000,1.380000,6.000000,0.558800,0.220000,6.000000,0.000120,0.000002,0.005500,0.000010,0.020000,0.000630,0.001100,0.009100,0.000840,0.000130,0.000147,1.700000e-06,2.328500e-03,0.214000,0.130000,0.230000,0.005000,0.098000,0.002000,0.000325,0.000160,1.245000e-05,5.500000e-05,24.000000,219.500000,9.000000,7.000000
75%,3.000000,0.000000,0.000000,1.649000e+03,881.500000,21.000000,7.400000,26.000000,11.000000,0.115000,0.005440,53.000000,17.800000,3.200000,11.000000,1.244600,0.490000,12.500000,0.000464,0.000005,0.012000,0.000036,0.030000,0.000933,0.001200,0.013300,0.001200,0.000170,0.000163,2.100000e-06,5.100000e-03,0.618000,0.330000,0.350000,0.008000,0.155000,0.005200,0.000789,0.001325,2.337500e-05,1.030000e-04,51.000000,485.000000,15.000000,16.000000
max,31.000000,2.000000,5.000000,3.251373e+06,1830.000000,380.000000,210.000000,82.000

In [172]:
food_filtered_fr.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,brands,brands_tags,categories,categories_tags,categories_fr,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_fr,cities_tags,purchase_places,stores,countries_tags,ingredients_text,allergens,traces_tags,additives_n,additives_tags,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states_tags,main_category_fr,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,pantothenic-acid_100g,potassium_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,selenium_100g,iodine_100g,fruits-vegetables-nuts_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
0,3087,http://world-fr.openfoodfacts.org/produit/0000000003087/farine-de-ble-noir-ferme-t-y-r-nao,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed,en:nutrition-facts-to-be-completed,en:ingredients-to-be-completed,en:expiration-date-to-be-completed,en:characteristics-to-be-completed,en:categories-to-be-completed,en:brands-completed,en:packaging-to-be-completed,en:quantity-completed,en:product-name-completed,en:photos-to-be-validated,en:photos-uploaded",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,24600,http://world-fr.openfoodfacts.org/produit/0000000024600/filet-de-boeuf,date-limite-app,1434530704,2015-06-17T08:45:04Z,1434535914,2015-06-17T10:11:54Z,Filet de bœuf,NaN,2.46 kg,NaN,NaN,Filet de bœuf,fr:filet-de-boeuf,Filet-de-boeuf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed,en:nutrition-facts-to-be-completed,en:ingredients-to-be-completed,en:expiration-date-completed,en:characteristics-to-be-completed,en:categories-completed,en:brands-to-be-completed,en:packaging-completed,en:quantity-completed,en:product-name-completed,en:photos-to-be-validated,en:photos-uploaded",Filet-de-boeuf,http://fr.openfoodfacts.org/images/products/000/000/002/4600/front.3.400.jpg,http://fr.openfoodfacts.org/images/products/000/000/002/4600/front.3.200.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,27205,http://world-fr.openfoodfacts.org/produit/0000000027205,tacinte,1458238630,2016-03-17T18:17:10Z,1458238638,2016-03-17T18:17:18Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed,en:nutrition-facts-to-be-completed,en:ingredients-to-be-completed,en:expiration-date-to-be-completed,en:characteristics-to-be-completed,en:categories-to-be-completed,en:brands-to-be-completed,en:packaging-to-be-completed,en:quantity-to-be-completed,en:product-name-to-be-completed,en:photos-to-be-validated,en:photos-uploaded",NaN,http://fr.openfoodfacts.org/images/products/000/000/002/7205/front.3.400.jpg,http://fr.openfoodfacts.org/images/products/000/000/002/7205/front.3.200.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN